In [40]:
import os
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.pipeline import Pipeline

In [ ]:
--------------------------------------------------------------------------------------------------
#THESE COULD BE TANGENTIAL TO A PIPELINE, BUT I HAVE WRITTEN A FUNCTION THAT LEVERAGES THEM BELOW
--------------------------------------------------------------------------------------------------

In [374]:
def convert_csv_into_nested_list(csv_path):
    df = pd.read_csv(csv_path)
    list_of_lists = df.values.tolist()
    return list_of_lists


def store_X(list_of_files, X_list):
    for fi in list_of_files:
        X_list.append(convert_csv_into_nested_list(os.path.join(path, fi)))
    return X_list
        
        
def convert_X_to_ndarray(X, list_object):
    for arr in list_object:
        X.append(np.array(arr))
    return X

In [ ]:
--------------------------------------------------------------------------
#THIS GUY IS FINNICKY, LET ME KNOW IF YOU SEE SOMETHING WRONG WITH MY CODE
--------------------------------------------------------------------------

In [375]:
def pad_values(X):
    X_pad = pad_sequences(X, dtype='float32',padding='post',value=-1000)
    return X_pad

In [41]:
-------
#NOTES
-------

#Robyn's functions for preprocessing
#Run third cell and all functions execute
#stores csvs in Features folder


#Function that takes in three coordinates as input and finds the angle between those three points
#note: there are points that were the exact position which created nan values for angles. example row 19 column 1 & 2 of 1.csv

def create_angle(x1, y1, x2, y2, x3, y3):
  vector_1 = [(x1-x2), (y1-y2)]
  vector_2 = [(x3-x2), (y3-y2)] 
  if x1 == x2 and y1 == y2:
    vector_1 = [1,0]
  if x3 == x2 and y3 == y2:
    vector_2 = [1,0]
  unit_vector_1 = vector_1 / np.linalg.norm(vector_1)
  unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
  dot_product = np.dot(unit_vector_1, unit_vector_2)
  angle = np.arccos(dot_product)
  #if angle = np.nan:
  #  angle = padded_value(-1000)
  return math.degrees(angle)

In [42]:
#Function that iterates through each row of a csv file and creates a list of angles
#key1, key2, key3 are the columns for the x coordinates (example: column 0,2,4)

def angles_list(csv_file, key_1, key_2, key_3):
  df = pd.read_csv(csv_file)
  list = [create_angle(df.loc[i][key_1], df.loc[i][key_1+1], df.loc[i][key_2], df.loc[i][key_2+1], df.loc[i][key_3], df.loc[i][key_3+1]) for i in range(len(df))]
  return list

In [43]:
#Function that 1. calls angle_list function for each of the 100 CSV files  2. appends that list to a new column in the CSV
def angle_column(list_key_1, list_key_2, list_key_3):
  csv_files = '../raw_data/CSV_Files'
  csv_features = '../raw_data/CSV_Features'
  for file in os.listdir(csv_files):
    #create a df from each csv file
    df = pd.read_csv(os.path.join(csv_files, file))
    i = 1
    #loop over every CSV file in this folder and run angles_list function on it
    for key_1, key_2, key_3 in zip(list_key_1, list_key_2, list_key_3):
      list_of_angles = angles_list(os.path.join(csv_files, file), key_1, key_2, key_3)
      #create a new column in the df and add each element of that list to that new df
      df[f'feature_{i}'] = list_of_angles
      i += 1    
    #convert df back to csv and save csvs in a file called csv_features
    df.to_csv(os.path.join(csv_features, file))

In [ ]:
---------------------------------------------------
#AUTOMATED CSV COLLECTION and X,y VARIABLE STORAGE
---------------------------------------------------

In [376]:
def temporary_preprocessing(good_or_training_path, bad_or_validation_path):
    '''this pads and stores X and generates a matching y'''
    
    # get the good features or training data
    X_list = []
    X = []
    list_of_files = os.listdir(good_path)
    store_X(list_of_files, X_list)
    convert_X_to_ndarray(X, X_list)
    X_good_features = X
    y_split_1 = len(X_good_features)
    
    #get the bad squats or validation data
    X_list = []
    X = []
    list_of_files = os.listdir(good_path)
    store_X(list_of_files, X_list)
    convert_X_to_ndarray(X, X_list)
    X_bad_features = X
    y_split_2 = len(X_good_features)
    
    #combine good and bad into ordered list so we know what's what
    X_good_and_bad_together = X_good_features + X_bad_features
    X_pad = pad_values(X_good_and_bad_together)
    
    #now make the y based on the x length
    y_good = [1 for i in range(1,y_split_1+1)]
    y_bad = [0 for i in range(1,y_split_2+1)]
    y = np.expand_dims((np.array(y_good+y_bad)),1) 
    
    return [X_pad,y]

In [ ]:
--------------------------------------------------
#HERE ARE THE INSTRUCTIONS FOR THE ABOVE FUNCTION
--------------------------------------------------

In [377]:
#just put in your your paths, example = "../raw_data/CSV_Features_good/"
good_path = 'INSERT_PATH_IN_QUOTES'
bad_path  = 'INSERT_PATH_IN_QUOTES'

#X and y values will automatically store
#IMPORTANT: you still have to scale the data 
X,y = temporary_preprocessing(good_path,bad_path)

In [ ]:
-------------
#OTHER NOTES
------------

In [ ]:
#1a. you can also store the features from today in a single line of code 
    #1b. uncomment and run this line: X_features = np.expand_dims(X[:,:,33],2)
#2. this could also work for train and val values
#3. not just splitting the 80 training CSVs 

In [ ]:
-----------------------------------------------------------------------------------
#I was doing above manually like this below, don't delete this please just in case 
-----------------------------------------------------------------------------------

In [335]:
#GOOD FEATURES

path = "../raw_data/CSV_Features_good/"
list_of_files = os.listdir("../raw_data/CSV_Features_good/")
X_list = []
X = []
X_pad = []

store_X(list_of_files, X_list)
convert_X_to_ndarray(X, X_list)
X_good_features = X

#BAD FEATURES

path = "../raw_data/CSV_Features_bad/"
list_of_files = os.listdir("../raw_data/CSV_Features_bad/")
X_list = []
X = []
X_pad = []

store_X(list_of_files, X_list)
convert_X_to_ndarray(X, X_list)
X_bad_features = X

In [ ]:
----------------------------------------------------------------------------------------
#THIS SECTION IS ME MANUALLY WORKING THROUGH THE CODE BEFORE I WROTE THE FUNCTION ABOVE
#I accidentally deleted some of it 
#but I got the X_good_and_bad_together the same way as the function
----------------------------------------------------------------------------------------

In [245]:
#this will store them in order
X_good_and_bad_together = X_good_features + X_bad_features

#you can check to see if they are ordered properly by printing the indexes pre and post
print(X_good_features[0][15][15])
print(X_bad_features[0][15][15])
print(X_good_and_bad_together[0][15][15])
print(X_good_and_bad_together[40][15][15])

780.0
630.0
780.0
630.0


In [248]:
#next we need to pad
#I can't get the pad function to call properly so I do it manually 
X_padded = pad_sequences(X_good_and_bad_together, dtype='float32',padding='post',value=-1000)

#check the shape
#should be 34 columns, 80 rows, length set by padder
X_padded.shape

(80, 176, 34)

In [251]:
#extract the last column from the arrays where the features are
X_final = X_padded[:,:,33]

#expand the dimensions so it can be trained by the rnn
X_final = np.expand_dims(X_final,axis=2)

In [253]:
#check the shape, should have 1 in the third dimension
X_final.shape

#now create the y variable 
#make the first half 1 and second half 0 according to how we added X_good and X_bad features together in the list 
y = np.full(80,1)
y[40:] = 0 

#check to make sure data is right
print(y[39],y[40],y.shape)

1 0 (80,)


In [254]:
#now expand the dimensions of y to (80,1) for the model
#I don't think this is necessary but me and jules did it during our ticket session
y = np.expand_dims(y,axis=1)
y.shape

(80, 1)

In [259]:
#finally let's scale on the fly

#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X_final)

#ERROR RECEIVED - ValueError: Found array with dim 3. StandardScaler expected <= 2. 

#well it looks like you can't scale a three dimensional array 
#so we have to scale it back
#we can do this by using an indexing trick

X_final = X_final[:, :, 0]
X_final.shape

(80, 176)

In [260]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_final)

In [264]:
print(X_scaled[0][0],X_final[0][0])

0.026414739 136.30002


In [266]:
#now we can move the dimensions back to 3
X_scaled = np.expand_dims(X_scaled, axis = 2)
X_scaled.shape

(80, 176, 1)

In [ ]:
------------
#FINAL NOTES
------------

#I'm using a simple RNN layar and I have been tweeking the parameters
#we need to experiment with more models 
#I haven't been able to get good accuracy, but I did improve the accuracy by jacking up the layers
#not above 50 percent though

------------
#TO DO LIST 
------------

#Make a pipeline perhaps incorporating some of the functions above
#however, since we have not had a thorough chance to test data / 
#we really can't determine which models would work best
#in that sense the pipeline is lower priority
#we should just be trying as many models as we can and then build up 
#I will admit that I dropped the ball and not getting a grid search set up today
#I am working on that right now

In [231]:
model = Sequential()
model.add(layers.SimpleRNN(500,input_dim=1, activation='relu'))
model.add(layers.Dense(250, activation='relu'))
model.add(layers.Dense(125, activation='relu'))
model.add(layers.Dense(75, activation='relu'))
model.add(layers.Dense(25, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [232]:
#fit the model
#may import implement early stopping for later
model_fit = model.fit(X,y_new)

3/3 [==============================] - 2s 176ms/step - loss: nan - accuracy: 0.5000


In [233]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, y_new, 
          epochs=75, 
          batch_size=32, 
          verbose=1,
          validation_split=0.2)

Epoch 1/75
2/2 [==============================] - 2s 498ms/step - loss: nan - accuracy: 0.3542 - val_loss: nan - val_accuracy: 1.0000
Epoch 2/75
2/2 [==============================] - 1s 394ms/step - loss: nan - accuracy: 0.3958 - val_loss: nan - val_accuracy: 1.0000
Epoch 3/75
2/2 [==============================] - 1s 283ms/step - loss: nan - accuracy: 0.3646 - val_loss: nan - val_accuracy: 1.0000
Epoch 4/75
2/2 [==============================] - 0s 265ms/step - loss: nan - accuracy: 0.3854 - val_loss: nan - val_accuracy: 1.0000
Epoch 5/75
2/2 [==============================] - 1s 441ms/step - loss: nan - accuracy: 0.3854 - val_loss: nan - val_accuracy: 1.0000
Epoch 6/75
2/2 [==============================] - 1s 297ms/step - loss: nan - accuracy: 0.3646 - val_loss: nan - val_accuracy: 1.0000
Epoch 7/75
2/2 [==============================] - 0s 276ms/step - loss: nan - accuracy: 0.3854 - val_loss: nan - val_accuracy: 1.0000
Epoch 8/75
2/2 [==============================] - 1s 271ms/ste

Epoch 62/75
2/2 [==============================] - 1s 316ms/step - loss: nan - accuracy: 0.3854 - val_loss: nan - val_accuracy: 1.0000
Epoch 63/75
2/2 [==============================] - 1s 515ms/step - loss: nan - accuracy: 0.4062 - val_loss: nan - val_accuracy: 1.0000
Epoch 64/75
2/2 [==============================] - 1s 482ms/step - loss: nan - accuracy: 0.3958 - val_loss: nan - val_accuracy: 1.0000
Epoch 65/75
2/2 [==============================] - 0s 315ms/step - loss: nan - accuracy: 0.3750 - val_loss: nan - val_accuracy: 1.0000
Epoch 66/75
2/2 [==============================] - 0s 261ms/step - loss: nan - accuracy: 0.3854 - val_loss: nan - val_accuracy: 1.0000
Epoch 67/75
2/2 [==============================] - 1s 368ms/step - loss: nan - accuracy: 0.3438 - val_loss: nan - val_accuracy: 1.0000
Epoch 68/75
2/2 [==============================] - 1s 505ms/step - loss: nan - accuracy: 0.3750 - val_loss: nan - val_accuracy: 1.0000
Epoch 69/75
2/2 [==============================] - 1s 3

In [ ]:
---------------------------------------------------------
#FINISH GRID SEARCH HERE AFTER BREAK, I'M HOPEFUL ROBYN!
--------------------------------------------------------